In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input
from tensorflow.keras.layers import Dense,Conv2D,GlobalAvgPool2D,Input
from tensorflow.keras.preprocessing.image import load_img,ImageDataGenerator
from tensorflow.keras import callbacks,optimizers
import numpy as np
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/all_fish/

/content/drive/MyDrive/all_fish


In [ ]:
#!unzip NEW.zip

Archive:  NEW.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of NEW.zip or
        NEW.zip.zip, and cannot find NEW.zip.ZIP, period.


In [ ]:
os.listdir("NEW")  

['fighter fish',
 'Gilt Head Bream',
 'jalebi fish',
 'Red hand fish',
 'cat fish',
 'Horse Mackerel',
 'gold fish',
 'Black Sea Sprat',
 'anglerfish',
 'european sea sturgeon',
 'tequils splitfin',
 'Red Sea Bream',
 'Striped Red Mullet',
 'sakhalin sturgeon',
 'smalltooth fish',
 'sea horse',
 'Sea Bass',
 'Shrimp',
 'Red Mullet',
 'tank cleaner fish',
 'Trout']

In [ ]:
!ls NEW/

 anglerfish		 'Horse Mackerel'     'sea horse'
'Black Sea Sprat'	 'jalebi fish'	       Shrimp
'cat fish'		 'Red hand fish'      'smalltooth fish'
'european sea sturgeon'  'Red Mullet'	      'Striped Red Mullet'
'fighter fish'		 'Red Sea Bream'      'tank cleaner fish'
'Gilt Head Bream'	 'sakhalin sturgeon'  'tequils splitfin'
'gold fish'		 'Sea Bass'	       Trout


In [ ]:
for i in os.listdir("NEW"):
  print(i,len(os.listdir("NEW/"+i)))

fighter fish 63
Gilt Head Bream 50
jalebi fish 52
Red hand fish 53
cat fish 52
Horse Mackerel 50
gold fish 100
Black Sea Sprat 50
anglerfish 54
european sea sturgeon 50
tequils splitfin 56
Red Sea Bream 50
Striped Red Mullet 50
sakhalin sturgeon 54
smalltooth fish 51
sea horse 50
Sea Bass 50
Shrimp 50
Red Mullet 50
tank cleaner fish 50
Trout 60


In [ ]:
try:
  os.mkdir("train")
  os.mkdir("test")
except:
  pass
for i in os.listdir("NEW"):
  try:
    os.mkdir("train/"+i)
    os.mkdir("test/"+i)
  except:
    pass
  for j in os.listdir("NEW/"+i)[:35]:
    os.rename("NEW/"+i+"/"+j,"train/"+i+"/"+j)
  for j in os.listdir("NEW/"+i)[:15]:
    os.rename("NEW/"+i+"/"+j,"test/"+i+"/"+j)

In [ ]:
def img_Data(dir_path,target_size,batch,class_lst,preprocessing,):
  if preprocessing:
    gen_object =ImageDataGenerator(preprocessing_function=preprocessing)
  else:
    gen_object =ImageDataGenerator()

  return(gen_object.flow_from_directory(dir_path,
                                          target_size= target_size,
                                          batch_size=batch,
                                          class_mode='sparse',
                                          classes=class_lst,
                                          shuffle=True))

In [ ]:
train_data_gen =img_Data("train",(224,224),500,os.listdir("train"),preprocess_input)
valid_data_gen =img_Data("test",(224,224),500,os.listdir("test"),preprocess_input)

Found 728 images belonging to 21 classes.
Found 314 images belonging to 21 classes.


In [ ]:
base_model=tf.keras.applications.mobilenet_v2.MobileNetV2(
              input_shape=(224,224,3),
              alpha=1.0,
              include_top=False,
              weights='imagenet',
              input_tensor=None,
              pooling=None,
              classes=1000,
              classifier_activation='softmax')

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.trainable=False

In [ ]:
model=tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAvgPool2D())
model.add(Dense(1024,activation='relu'))
model.add(Dense(1024,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
elst=callbacks.EarlyStopping(monitor='val_loss',patience=5,mode='min')
save_ck=callbacks.ModelCheckpoint('.mdl_wt.hdf5',save_best_only=True,monitor='val_loss',mode='min')

In [ ]:
model.fit(train_data_gen,batch_size=500,validation_data=valid_data_gen,callbacks=[elst,save_ck],epochs=10)

Epoch 1/10
2/2 [==============================] - 218s 137s/step - loss: 5.6564 - accuracy: 0.2308 - val_loss: 3.0056 - val_accuracy: 0.4459
Epoch 2/10
2/2 [==============================] - 53s 40s/step - loss: 2.0605 - accuracy: 0.6209 - val_loss: 1.1752 - val_accuracy: 0.7930
Epoch 3/10
2/2 [==============================] - 52s 26s/step - loss: 0.8950 - accuracy: 0.8681 - val_loss: 0.7147 - val_accuracy: 0.8185
Epoch 4/10
2/2 [==============================] - 47s 34s/step - loss: 0.4647 - accuracy: 0.8984 - val_loss: 0.4697 - val_accuracy: 0.8471
Epoch 5/10
2/2 [==============================] - 48s 23s/step - loss: 0.2774 - accuracy: 0.9354 - val_loss: 0.4162 - val_accuracy: 0.8758
Epoch 6/10
2/2 [==============================] - 53s 40s/step - loss: 0.2002 - accuracy: 0.9423 - val_loss: 0.2937 - val_accuracy: 0.9076
Epoch 7/10
2/2 [==============================] - 53s 39s/step - loss: 0.1420 - accuracy: 0.9533 - val_loss: 0.2434 - val_accuracy: 0.9172
Epoch 8/10
2/2 [=========